# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
%%sql


In [12]:
# Load the sms_spam dataset
# See: https://huggingface.co/datasets/sms_spam

from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]


# View the dataset characteristics

In [13]:
for split in splits:
    dataset[split] = dataset[split].shuffle(seed=42).select(range(len(dataset[split])//5))

In [14]:
print(f"The train dataset is of type: {type(dataset['train'])}")
print(f"The length of the train dataset is: {len(dataset['train'])}")
print(f"The length of the test dataset is: {len(dataset['test'])}")
print(f"Here is one sample from the train dataset: {dataset['train'][0]=}")

The train dataset is of type: <class 'datasets.arrow_dataset.Dataset'>
The length of the train dataset is: 891
The length of the test dataset is: 223
Here is one sample from the train dataset: dataset['train'][0]={'sms': "How would my ip address test that considering my computer isn't a minecraft server\n", 'label': 0}


In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
print(f"The tokenized train dataset is of type: {type(tokenized_dataset['train'])}")
print(f"The length of the tokenized train dataset is: {len(tokenized_dataset['train'])}")
print(f"Here is one sample from the train dataset: {tokenized_dataset['train'][0]=}")

/home/tumaaa/work/repos/udacity_nano_degree_generative_ai/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 223/223 [00:00<00:00, 16414.77 examples/s]

The tokenized train dataset is of type: <class 'datasets.arrow_dataset.Dataset'>
The length of the tokenized train dataset is: 891
Here is one sample from the train dataset: tokenized_dataset['train'][0]={'sms': "How would my ip address test that considering my computer isn't a minecraft server\n", 'label': 0, 'input_ids': [101, 2129, 2052, 2026, 12997, 4769, 3231, 2008, 6195, 2026, 3274, 3475, 1005, 1056, 1037, 3067, 10419, 8241, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.parameters():
    param.requires_grad = False

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


In [28]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
def create_trainer(model, tokenizer, tokenized_dataset):
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir="./data/sms_spam_classifier",
            learning_rate=2e-3,
            # Reduce the batch size if you don't have enough memory
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            num_train_epochs=1,
            weight_decay=0.01,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
        ),
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics,
    )
    return trainer

In [29]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

trainer = create_trainer(model, tokenizer, tokenized_dataset)
trainer.evaluate()


{'eval_loss': 0.6838708519935608,
 'eval_accuracy': 0.7488789237668162,
 'eval_runtime': 6.8028,
 'eval_samples_per_second': 32.781,
 'eval_steps_per_second': 8.232}

In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [26]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [27]:
from peft import LoraConfig, get_peft_model, TaskType
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
    inference_mode=False
)
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()


trainable params: 1,479,172 || all params: 67,842,052 || trainable%: 2.180317305260755


In [31]:
trainer_lora = create_trainer(lora_model, tokenizer, tokenized_dataset)
trainer_lora.evaluate()

{'eval_loss': 0.6838708519935608,
 'eval_accuracy': 0.7488789237668162,
 'eval_runtime': 9.5239,
 'eval_samples_per_second': 23.415,
 'eval_steps_per_second': 5.88}

In [37]:
trainer_lora.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.050453,0.986547


TrainOutput(global_step=223, training_loss=0.3088602656206208, metrics={'train_runtime': 66.5083, 'train_samples_per_second': 13.397, 'train_steps_per_second': 3.353, 'total_flos': 10373903680320.0, 'train_loss': 0.3088602656206208, 'epoch': 1.0})

In [38]:
trainer_lora.evaluate()

{'eval_loss': 0.05045327916741371,
 'eval_accuracy': 0.9865470852017937,
 'eval_runtime': 6.8661,
 'eval_samples_per_second': 32.478,
 'eval_steps_per_second': 8.156,
 'epoch': 1.0}

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.